In [1]:
#!git clone https://github.com/htuannn/SpineXR-Lesions-Detection/tree/main

Cloning into 'SpineXR-YoloV5'...
remote: Enumerating objects: 127, done.
remote: Counting objects: 100% (127/127), done.
remote: Compressing objects: 100% (91/91), done.
remote: Total 127 (delta 32), reused 119 (delta 29), pack-reused 0
Receiving objects: 100% (127/127), 2.29 MiB | 23.73 MiB/s, done.
Resolving deltas: 100% (32/32), done.


#Prepare

In [9]:
#%cd SpineXR-Lesions-Detection/YOLOv5

/content/SpineXR-YoloV5/YOLOv5


In [6]:
#Install requirements
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.2/611.2 kB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.9 MB/s eta 0:00:00


In [3]:
#get data
!gdown 1-3JOV67ee7JmyRFiaHq6jEgkw4j6JyGo
!unzip data.zip -d data

Streaming output truncated to the last 5000 lines.
  inflating: content/data/labels/test/d4dd6e89a73ce4c1d7f16562b535df16.txt  
  inflating: content/data/labels/test/998554daa9553b1d76e582da9ab0416c.txt  
  inflating: content/data/labels/test/2cfb0d8b66f4b7452e6fe7ba090d7ca0.txt  
  inflating: content/data/labels/test/a6380559f7e630b0b1f1fbdeb4a85fb3.txt  
  inflating: content/data/labels/test/e4122d6b1efc7266329f0ab5db456924.txt  
  inflating: content/data/labels/test/4481ae5cfe33c0958bae66c5ba672b58.txt  
  inflating: content/data/labels/test/cdd5f1cc9bdd42a9ee7e106496dc9595.txt  
  inflating: content/data/labels/test/9192a01e564966cf512a1770648048dd.txt  
  inflating: content/data/labels/test/cacd20102ae5ca48207211c5f4c0ec60.txt  
  inflating: content/data/labels/test/be6b4dd7edffa814c0d1e35e50099eb6.txt  
  inflating: content/data/labels/test/e56b5bc0a1715a6114b8f166e4154306.txt  
  inflating: content/data/labels/test/24f2987c0cb86c2ff9f6efa0ae43d7f0.txt  
  inflating: content/data

#Define training data

In [43]:
%cd /content/SpineXR-YoloV5/YOLOv5

/content/SpineXR-YoloV5/YOLOv5


In [44]:
ls

best.pt           dicom2png.py  models/       requirements.txt      train.py
convert_dicom.sh  export.py     __pycache__/  runs/                 utils/
data/             hubconf.py    README.md     SpineXR_YoloV5.ipynb  val.py


In [46]:
from os import listdir
from os.path import isfile, join
import yaml
from glob import glob

cwd = 'data'

with open(join( cwd , 'train.txt'), 'w') as f:
    for path in glob('content/data/images/train/*'):
        f.write(path+'\n')

with open(join( cwd , 'val.txt'), 'w') as f:
    for path in glob('content/data/images/val/*'):
        f.write(path+'\n')

with open(join( cwd , 'test.txt'), 'w') as f:
    for path in glob('content/data/images/test/*'):
        f.write(path+'\n')

CLASSES = ["Disc space narrowing", "Foraminal stenosis", "Osteophytes", "Other lesions", "Spondylolysthesis", "Surgical implant", "Vertebral collapse"]

data = dict(
    train =  join( cwd , 'train.txt') ,
    val   =  join( cwd , 'val.txt' ),
    test  =  join( cwd , 'test.txt' ),
    nc    = len(CLASSES),
    names = CLASSES
    )

with open(join( cwd , 'vinbigdata.yaml'), 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)

f = open(join( cwd , 'vinbigdata.yaml'), 'r')
print('\nyaml:')
print(f.read())


yaml:
names:
- Disc space narrowing
- Foraminal stenosis
- Osteophytes
- Other lesions
- Spondylolysthesis
- Surgical implant
- Vertebral collapse
nc: 7
test: data/test.txt
train: data/train.txt
val: data/val.txt



#Training

**Selecting Models**\
In this notebok I'm using v5x. To select your prefered model just replace
*--cfg models/yolov5x.yaml --weights yolov5x.pt* with the following command:\
v5s : --cfg models/yolov5s.yaml --weights yolov5s.pt\
v5m : --cfg models/yolov5m.yaml --weights yolov5m.pt\
v5l : --cfg models/yolov5l.yaml --weights yolov5l.pt\
v5x : --cfg models/yolov5x.yaml --weights yolov5x.pt

In [ ]:
!WANDB_MODE="dryrun" python train.py --img 640 --batch 16 --epochs 45 --data data/vinbigdata.yaml --cfg models/yolov5x.yaml --weights yolov5x.pt --cache

In [ ]:
#For resume training
!WANDB_MODE="dryrun" python train.py --img 640 --batch 16 --epochs 45 --data data/vinbigdata.yaml --weights runs/train/exp3/weights/last.pt --cache --resume

train: weights=/content/SpineXR-YoloV5/runs/train/exp3/weights/last.pt, cfg=, data=data/vinbigdata.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=45, batch_size=16, imgsz=640, rect=False, resume=True, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 2659 commits. Use 'git pull ultralytics master' or 'git clone https://github.com/ultralytics/yolov5' to update.
YOLOv5 🚀 42d9eaf Python-3.10.11 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, 

#Evaluate

In [47]:
#inite new data yaml file for evaluate
from os import listdir
from os.path import isfile, join
import yaml
from glob import glob

cwd = 'data'

CLASSES = ["Disc space narrowing", "Foraminal stenosis", "Osteophytes", "Other lesions", "Spondylolysthesis", "Surgical implant", "Vertebral collapse"]

data = dict(
    train =  join( cwd , 'train.txt') ,
    val   =  join( cwd , 'test.txt' ),
    nc    = len(CLASSES),
    names = CLASSES
    )

with open(join( cwd , 'vinbigdata_1.yaml'), 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)

f = open(join( cwd , 'vinbigdata_1.yaml'), 'r')
print('\nyaml:')
print(f.read())


yaml:
names:
- Disc space narrowing
- Foraminal stenosis
- Osteophytes
- Other lesions
- Spondylolysthesis
- Surgical implant
- Vertebral collapse
nc: 7
train: data/train.txt
val: data/test.txt



In [48]:
#eval model
!python val.py --weights best.pt --data data/vinbigdata_1.yaml --img 640 --max-det 101

val: data=data/vinbigdata_1.yaml, weights=['best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=101, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 2023-6-18 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 322 layers, 86213788 parameters, 0 gradients, 203.9 GFLOPs
val: Scanning /content/SpineXR-YoloV5/YOLOv5/data/test... 873 images, 0 backgrounds, 0 corrupt: 100% 873/873 [00:00<00:00, 1006.35it/s]
val: WARNING ⚠️ data/images/test/078075701bb83b1ac2f833c0576a33fd.jpg: 2 duplicate labels removed
val: WARNING ⚠️ data/images/test/0881a2daafeb7f9514bd038be28e1731.jpg: 2 duplicate labels removed
val: WARNING ⚠️ data/images/test/0ccf49e003ad267062de0c14bb4d4088.jpg: 4 duplicate labels removed
val: WARNING ⚠️ data/images/test/0eb0109248f23d97ef84f695

#Predict

In [ ]:
!python detect.py --weights runs/train/exp3/weights/best.pt --source "/content/content/data/images/test/*.jpg"

detect: weights=['/content/drive/MyDrive/AI/SpineXR_yolov5/exp3/weights/best.pt'], source=/content/content/data/images/test/*.jpg, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
requirements: Ultralytics requirement "gitpython>=3.1.30" not found, attempting AutoUpdate...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 292.2 MB/s eta 0:00:00

requirements: 1 package updated per /content/SpineXR-YoloV5/requirements.txt
requirements: ⚠️ Restart runtime or rerun

In [ ]:
!cp -r /content/SpineXR-YoloV5/runs/detect/exp /content/drive/MyDrive/AI/SpineXR_yolov5/test